# Model 1 (LGMBRegressor)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import lightgbm as lgbm

# Data Prep

In [ ]:
# import data


In [ ]:
train_data = train_logs.merge(train_scores, how='inner', on="id")

In [ ]:
train_data

In [ ]:
len(train_data["id"].unique())

In [ ]:
test_logs

In [ ]:
train_data["activity"].describe()

In [ ]:
train_data[train_data["id"]=="001519c8"]

In [ ]:
print(train_data["activity"].unique())

In [ ]:
def feature_eng(df) :
    df_train  = df[["id","down_time",]].groupby("id").min()
    df_temp   = df[["id","up_time",]].groupby("id").max()
    df_train  = pd.merge(df_train, df_temp, on="id", how="left")
    df_temp   = df[["id","action_time",]].groupby("id").sum()
    df_train  = pd.merge(df_train, df_temp, on="id", how="left")
    df_temp   = df[["id","word_count",]].groupby("id").max()
    df_train  = pd.merge(df_train, df_temp, on="id", how="left")
        
    df["activity"] = df["activity"].str[:4]
    df["Inpu"] = df["activity"].apply(lambda x:1 if x=="Inpu" else 0)
    df["Move"] = df["activity"].apply(lambda x:1 if x=="Move" else 0)
    df["Nonp"] = df["activity"].apply(lambda x:1 if x=="Nonp" else 0)
    df["Past"] = df["activity"].apply(lambda x:1 if x=="Past" else 0)
    df["Remo"] = df["activity"].apply(lambda x:1 if x=="Remo" else 0)
    df["Repl"] = df["activity"].apply(lambda x:1 if x=="Repl" else 0)
    
    df_temp = df[["id","Inpu","Move","Nonp","Past","Remo","Repl"]].groupby("id").sum()
    df_train  = pd.merge(df_train, df_temp, on="id", how="left")
    
    return df_train

In [ ]:
train_df = feature_eng(train_data)

In [ ]:
train_df

In [ ]:
train_df.drop("id", axis= 1, inplace = True)

In [ ]:
model = lgbm.LGBMRegressor(n_estimators = 500)
x = df
y = train_scores

In [ ]:
model.fit(x,y)

In [ ]:
lgbm.plot_importance(model, importance_type="gain")

# Submission

In [ ]:
df_test = feature_eng(test_logs)

In [ ]:
df_test.drop("id", axis = 1, inplace = True)

In [ ]:
predicted_scores = model.predict(df_test)

In [ ]:
submissions_v1 = pd.DataFrame({'id': df_test['id'], 'score':predicted_scores})